In [9]:
from stable_baselines3 import PPO, SAC, DDPG, A2C
from stable_baselines3.common.env_util import make_vec_env
from env import CropEnv, nebraska_maize_config 
from utils import calc_eto_faopm, evaluate_agent, evaluate_agent_single_year

In [20]:
gendf=calc_eto_faopm('data/CDWG.dat',1995,40.4,1072,True,["simyear","jday","minTemp","maxTemp","precip","rad"])
config = nebraska_maize_config
config['gendf'] = gendf
config['crop'] = "Rice"
env = CropEnv(config) 

In [17]:
#Parallel environments

model = PPO("MlpPolicy", env, n_steps=500)
#model = PPO.load("ppo")
#model.set_env(env)
model.learn(total_timesteps=250000, progress_bar=True)
model.save(".saved_model/ppo_500")

Output()

/opt/anaconda3/envs/rl-agri/lib/python3.9/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
/opt/anaconda3/envs/rl-agri/lib/python3.9/site-packages/stable_baselines3/ppo/ppo.py:155: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 500`, after every 7 untruncated mini-batches, there will be a truncated mini-batch of size 52
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=500 and n_envs=1)
  warnings.warn(


Traceback (most recent call last):

File "/opt/anaconda3/envs/rl-agri/lib/python3.9/site-packages/tqdm/std.py", line 1148, in __del__

self.close()

File "/opt/anaconda3/envs/rl-agri/lib/python3.9/site-packages/tqdm/std.py", line 1277, in close

if self.last_print_t < self.start_t + self.delay:

AttributeError: 'tqdm' object has no attribute 'last_print_t'

In [7]:
#Parallel environments

model = SAC("MlpPolicy", env, train_freq=500)
#model = PPO.load("ppo")
#model.set_env(env)
model.learn(total_timesteps=250000, progress_bar=True)
model.save(".saved_model/ppo_500")

Output()

/opt/anaconda3/envs/rl-agri/lib/python3.9/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


KeyboardInterrupt: 

In [3]:
model = DDPG("MlpPolicy", env)
#model = PPO.load("ppo")
#model.set_env(env)
model.learn(total_timesteps=250000, progress_bar=True)
model.save(".saved_model/ddpg")

/opt/anaconda3/envs/rl-agri/lib/python3.9/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Output()

In [9]:
model = A2C("MlpPolicy", env)
model.learn(total_timesteps=250000, progress_bar=True)
model.save(".saved_model/a2c")

Output()

/opt/anaconda3/envs/rl-agri/lib/python3.9/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [18]:
model = PPO.load(".saved_model/ppo_500")
evaluate_agent(model, CropEnv, config, stable_baselines=True)

100%|██████████| 100/100 [00:06<00:00, 16.12it/s]


(-0.07612373086856326, -0.07694350728858997)